## Import Library and Load data

In [1]:
#Modifikasi
import warnings
import zipfile
import numpy as np
import pandas as pd
from pathlib import Path
pd.set_option('display.max_columns', 100)

#Perhitungan
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt

#Imputasi
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


# Modeling
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV, GroupKFold,KFold, TimeSeriesSplit   
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc,roc_auc_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, RepeatVector, TimeDistributed

#Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Feature Importance
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
train = pd.read_csv('raw_dataset/Train.csv')
test = pd.read_csv('raw_dataset/Test.csv')

## Data Cleaning

In [3]:
train['Date'] = pd.to_datetime(train['Date'], dayfirst=True, errors='coerce')

Date = train.copy()
Date['Date'] = pd.to_datetime(Date['Date'])


# 3. Menetapkan kolom Date sebagai index
train.set_index('Date', inplace=True)

C:\Users\booma\AppData\Local\Temp\ipykernel_24164\2488958140.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['Date'] = pd.to_datetime(train['Date'], dayfirst=True, errors='coerce')


In [4]:
test['Date'] = pd.to_datetime(test['Date'], dayfirst=True, errors='coerce')

Date_test = test.copy()
Date_test['Date'] = pd.to_datetime(Date['Date'])


# 3. Menetapkan kolom Date sebagai index
test.set_index('Date', inplace=True)

C:\Users\booma\AppData\Local\Temp\ipykernel_24164\1334361193.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test['Date'] = pd.to_datetime(test['Date'], dayfirst=True, errors='coerce')


## Imputasi

In [5]:
# df = train.copy()
# df.drop(columns=['ID_Zindi','ID'],inplace=True)
# test.drop(columns=['ID_Zindi','ID'],inplace=True)

# def impute_missing_values(df, cols_to_impute, drop_cols=['LAT', 'LON'], n_estimators=100, random_state=42):
#     for col in cols_to_impute:
#         if df[col].isna().sum() > 0:  # Cek apakah ada nilai NaN pada kolom
#             non_missing_data = df[df[col].notna()]  # Data tanpa nilai NaN untuk training
#             X_train = non_missing_data.drop(columns=[col] + drop_cols)  # Fitur training tanpa kolom target
#             y_train = non_missing_data[col]  # Target untuk training
            
#             # Inisiasi Random Forest Regressor dan training
#             rf_imputer = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
#             rf_imputer.fit(X_train, y_train)
            
#             # Melakukan prediksi untuk mengisi nilai NaN
#             X_pred = df[df[col].isna()].drop(columns=[col] + drop_cols)
#             df.loc[df[col].isna(), col] = rf_imputer.predict(X_pred)
    
#     return df


# cols_to_impute_rf = ['AAI', 'CloudFraction','LST', 'NO2_trop', 'NO2_strat', 'NO2_total', 'TropopausePressure']
# df = impute_missing_values(df, cols_to_impute_rf)
# test = impute_missing_values(test, cols_to_impute_rf)

# # Imputasi untuk kolom dengan missing data sedikit (Mean Imputation)
# cols_to_impute_mean = ['GT_NO2']
# mean_imputer = SimpleImputer(strategy='mean')
# df[cols_to_impute_mean] = mean_imputer.fit_transform(df[cols_to_impute_mean])

# # Time series imputation using Iterative Imputer (Multiple Imputation)
# time_series_cols = ['Precipitation']
# time_series_imputer = IterativeImputer(random_state=42)
# df[time_series_cols] = time_series_imputer.fit_transform(df[time_series_cols])
# test[time_series_cols] = time_series_imputer.fit_transform(test[time_series_cols])

# Feature Engineering

In [24]:
# df.to_csv('train_imputed_with_rf_regressor.csv', index=False)
# test.to_csv('test_imputed_with_rf_regressor.csv', index=False)

data = pd.read_csv('final_dataset/train_imputed_with_rf_regressor.csv')
dtest = pd.read_csv('final_dataset/test_imputed_with_rf_regressor.csv')

data.drop(['LAT','LON'],axis=1,inplace=True)
dtest.drop(['LAT','LON'],axis=1,inplace=True)

In [25]:
kmeans = KMeans(n_clusters=3)
data['Kmeans'] = kmeans.fit_predict(data[['NO2_strat', 'NO2_total', 'NO2_trop']])

In [26]:
scaler = StandardScaler()
data['TropopausePressure'] = scaler.fit_transform(data[['TropopausePressure']])

In [ ]:
# Misalnya, data berisi kolom 'Precipitation', 'LST', 'AAI'
poly = PolynomialFeatures(degree=2)

# Menerapkan transformasi polinomial pada kolom-kolom yang ditentukan
poly_features = poly.fit_transform(data[['Precipitation', 'LST', 'AAI']])
poly_features_test = poly.fit_transform(dtest[['Precipitation', 'LST', 'AAI']])

# Membuat DataFrame baru dengan nama kolom yang sesuai
poly_feature_columns = poly.get_feature_names_out(['Precipitation', 'LST', 'AAI'])

# Menggabungkan hasilnya dengan data asli (jika perlu)
poly_data = pd.DataFrame(poly_features, columns=poly_feature_columns)

poly_data.drop(['1','Precipitation','LST','AAI',],axis=1,inplace=True)

In [ ]:
result = pd.concat([data, poly_data], axis=1)

In [29]:
result['GT_NO2'] = pd.to_numeric(result['GT_NO2'], errors='coerce')  # Pastikan target numerik

In [36]:
target_col = 'GT_NO2'  # Nama kolom target
cols = [col for col in result.columns if col != target_col] + [target_col]
result = result[cols]

# Model Creation

In [37]:
result

,Precipitation,LST,AAI,CloudFraction,NO2_strat,NO2_total,NO2_trop,TropopausePressure,Kmeans,Precipitation^2,Precipitation LST,Precipitation AAI,LST^2,LST AAI,AAI^2,GT_NO2
0,0.000000,277.4618,0.230527,0.559117,0.000024,0.000117,0.000163,-0.858776,0,0.000000,0.000000,0.000000,76985.050459,63.962302,0.053142,31.000
1,3.047342,277.9274,-0.074006,0.869309,0.000024,0.000127,0.000123,-0.858437,0,9.286295,846.939922,-0.225522,77243.639671,-20.568365,0.005477,42.000
2,0.000000,277.1622,0.024470,0.674160,0.000024,0.000086,0.000089,-0.859968,0,0.000000,0.000000,0.000000,76818.885109,6.782106,0.000599,31.000
3,1.200467,277.4052,-0.010442,0.920054,0.000024,0.000124,0.000123,-0.858770,0,1.441121,333.015752,-0.012535,76953.644987,-2.896627,0.000109,30.000
4,1.274564,278.9034,-0.176178,0.747464,0.000024,0.000116,0.000164,-0.859480,0,1.624513,355.480174,-0.224550,77787.106532,-49.136681,0.031039,58.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86579,0.000000,282.6682,-0.434350,0.250490,0.000032,0.000643,0.000579,-1.336112,2,0.000000,0.000000,-0.000000,79901.311291,-122.776903,0.188660,39.750
86580,0.000000,272.3992,-1.479808,0.402424,0.000025,0.000101,0.000077,0.098240,0,0.000000,0.000000,-0.000000,74201.324161,-403.098429,2.189831,30.125
86581,0.000000,284.9800,-0.157753,0.000000,0.000031,0.000153,0.000122,-1.340838,0,0.000000,0.000000,-0.000000,81213.600400,-44.956553,0.024886,28.325
86582,0.000000,283.1990,-0.798636,0.399524,0.000031,0.000717,0.000606,-1.336939,2,0.000000,0.000000,-0.000000,80201.673601,-226.173003,0.637820,21.250


In [31]:
def create_sequences(data, seq_length, output_length=1):
    X, y = [], []
    for i in range(len(data) - seq_length - output_length + 1):
        X.append(data[i:i+seq_length])  # Ambil input sequence
        y.append(data[i+seq_length:i+seq_length+output_length])  # Output beberapa langkah ke depan
    return np.array(X), np.array(y)

In [32]:
from sklearn.ensemble import HistGradientBoostingRegressor


In [45]:
seq_length = 15
output_length = 1  # Prediksi 1 langkah ke depan
X, y = create_sequences(result['GT_NO2'].values, seq_length, output_length)

# Split result menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Model SGBRegressor
model = HistGradientBoostingRegressor(loss="squared_error", max_iter=100)
model.fit(X_train, y_train.ravel())  # `.ravel()` agar sesuai dengan input

# Prediksi
y_pred = model.predict(X_test)

# Evaluasi
from sklearn.metrics import root_mean_squared_error 
rmse = root_mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

MSE: 10.05865473721987


In [ ]:
import joblib
# Simpan model machine learning
joblib.dump(model, 'model.pkl')

# Simpan scaler
joblib.dump(scaler, 'scaler.pkl')

# Simpan KMeans
joblib.dump(kmeans, 'kmeans.pkl')

# Simpan PolynomialFeatures
joblib.dump(poly, 'poly.pkl')

print("Model dan preprocessing berhasil disimpan!")


['model.pkl']

In [44]:
X

array([[31.   , 42.   , 31.   , ..., 58.   , 26.   , 38.   ],
       [42.   , 31.   , 30.   , ..., 26.   , 38.   , 34.   ],
       [31.   , 30.   , 58.   , ..., 38.   , 34.   , 41.   ],
       ...,
       [29.225, 27.675, 19.875, ..., 56.275, 39.75 , 30.125],
       [27.675, 19.875, 24.225, ..., 39.75 , 30.125, 28.325],
       [19.875, 24.225, 56.275, ..., 30.125, 28.325, 21.25 ]])